# Creating VGG-11 from VGG-16 in Pytorch
### Code written following this tutorial blog: https://blog.paperspace.com/vgg-from-scratch-pytorch/
### VGG paper https://arxiv.org/pdf/1409.1556.pdf?ref=blog.paperspace.com

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

# DirectML for AMD compatibility - this is way worse than cuda...
import torch_directml

In [2]:
# Device configuration
device = torch_directml.device(torch_directml.default_device())
print(device)

privateuseone:0


## Data Loaders
### Using the CIFAR dataset

In [3]:
# Create a data loader for handling the CIFAR data
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):

    # mean and std of the rgb values in the images
    normalize = transforms.Normalize( # tutorial claimed these values available online
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    transform = transforms.Compose([
            transforms.Resize((227, 227)), # in vgg paper, images are 224x224
            transforms.ToTensor(),
            normalize,
    ])

    ### Test Data ###
    if test:
        dataset = datasets.CIFAR100(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    ### Training and Validation Data ###
    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

In [9]:
# Data loaders
train_loader, valid_loader = data_loader(data_dir='../data',
                                         batch_size=32)

test_loader = data_loader(data_dir='../data',
                              batch_size=32,
                              test=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# VGG-11

In [10]:
# Design the model's neural network
class VGG11(nn.Module):

    def __init__(self, num_classes=10):
        super(VGG11, self).__init__()
        
        # Convolution layers #
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())

        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

        self.layer6 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())

        self.layer8 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))

        # Fully connected layers #
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7 * 7 * 512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())

        # Classification Layer #
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    # Define how the layers are connected
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = self.layer5(out)
      out = self.layer6(out)
      out = self.layer7(out)
      out = self.layer8(out)
      out = out.reshape(out.size(0), -1)
      out = self.fc(out)
      out = self.fc1(out)
      out = self.fc2(out) # classification
      return out

## Training the VGG model

In [14]:
### Hyper Parameters ###

num_classes = 100
num_epochs = 10
learning_rate = 0.005

model = VGG11(num_classes).to(device) # use GPU


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)

In [15]:
total_params = sum(p.numel() for p in model.parameters())
print(f'Total number of parameters in this implementation of VGG-11: {total_params}')

Total number of parameters in this implementation of VGG-11: 129181540


In [16]:
# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/10], Step [1/1407], Loss: 4.6610
Epoch [1/10], Step [2/1407], Loss: 4.6068
Epoch [1/10], Step [3/1407], Loss: 4.7159
Epoch [1/10], Step [4/1407], Loss: 4.6608
Epoch [1/10], Step [5/1407], Loss: 4.6425
Epoch [1/10], Step [6/1407], Loss: 4.6121
Epoch [1/10], Step [7/1407], Loss: 4.6351
Epoch [1/10], Step [8/1407], Loss: 4.6264
Epoch [1/10], Step [9/1407], Loss: 4.7980
Epoch [1/10], Step [10/1407], Loss: 4.7045
Epoch [1/10], Step [11/1407], Loss: 4.6449
Epoch [1/10], Step [12/1407], Loss: 4.6093
Epoch [1/10], Step [13/1407], Loss: 4.7841
Epoch [1/10], Step [14/1407], Loss: 4.7966
Epoch [1/10], Step [15/1407], Loss: 4.6695
Epoch [1/10], Step [16/1407], Loss: 4.7100
Epoch [1/10], Step [17/1407], Loss: 4.7208
Epoch [1/10], Step [18/1407], Loss: 4.7132
Epoch [1/10], Step [19/1407], Loss: 4.7184
Epoch [1/10], Step [20/1407], Loss: 4.8089
Epoch [1/10], Step [21/1407], Loss: 4.5907
Epoch [1/10], Step [22/1407], Loss: 4.6816
Epoch [1/10], Step [23/1407], Loss: 4.6421
Epoch [1/10], Step [

## Testing

In [17]:
# Evaluation on the test dataset
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Accuracy of the network on the 10000 test images: 48.23 %


# Results
### CIFAR-10 228x228 10 epochs
    ~80% accuracy
### CIFAR-10 32x32 10 epochs
    ~75% accuracy
### CIFAR-10 64X64 10 epochs
    ~80% accuracy
### CIFAR-10 128x128 10 epochs
    ~82% accuracy
### CIFAR-100 32x32 10 epochs
    ~40% accuracy
### CIFAR-100 32X32 25 epochs
    ~41% accuracy
### CIFAR-100 64x64 25 epochs
    ~48% accuracy
### CIFAR-100 228x228 10 epochs
    ~48% accuracy